In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession.builder.appName('cruise_model').getOrCreate()

In [3]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [5]:
data.show()
data.printSchema()
data.columns

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_lineIndex")

In [15]:
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_lineIndex|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|            16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|            16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|             1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|             1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|             1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.

In [16]:
Asem = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers','length', 'cabins','passenger_density','Cruise_lineIndex']
                       , outputCol='features')

In [21]:
output = Asem.transform(indexed)
output.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [22]:
final_data = output.select('features','crew')

In [23]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [26]:
from pyspark.ml.regression import LinearRegression


In [27]:
lr = LinearRegression(labelCol='crew')

In [28]:
lr_model = lr.fit(train_data)

In [29]:
test_results = lr_model.evaluate(test_data)

In [30]:
test_results.rootMeanSquaredError

0.7347169949030269

In [31]:
test_results.r2

0.9590971313923421

In [32]:
unlabeled_data = test_data.select('features')

In [34]:
prediction = lr_model.transform(unlabeled_data)

In [35]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.167538417808146|
|[5.0,86.0,21.04,9...| 9.224166853537735|
|[5.0,122.0,28.5,1...|  5.51204124970504|
|[6.0,113.0,37.82,...|11.592104512272025|
|[6.0,158.0,43.7,1...|13.755421544592124|
|[8.0,77.499,19.5,...| 8.598170941030748|
|[9.0,113.0,26.74,...| 11.37544254761053|
|[10.0,46.0,7.0,6....| 2.545900946652371|
|[10.0,77.0,20.16,...| 8.800882423543026|
|[10.0,86.0,21.14,...| 9.679210317059434|
|[10.0,90.09,25.01...| 8.753306689065534|
|[11.0,90.0,22.4,9...|10.059440994062312|
|[11.0,90.09,25.01...| 8.744396292986055|
|[11.0,91.0,20.32,...| 9.128338697222532|
|[11.0,138.0,31.14...|13.021666899419596|
|[12.0,42.0,14.8,7...| 6.666325211858128|
|[12.0,91.0,20.32,...| 9.119428301143053|
|[12.0,138.0,31.14...| 13.01275650334012|
|[13.0,25.0,3.82,5...| 2.808655679658184|
|[13.0,63.0,14.4,7...| 6.638597812571679|
+--------------------+------------